In [1]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer, LancasterStemmer, SnowballStemmer
from nltk.corpus import wordnet
from nltk.tokenize import PunktSentenceTokenizer # unsupervised machine learning sentence tokenizer , can be trained, but we use the default
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()
dataset = pd.read_csv('files/train.csv')
data_list =[]
data_list2 =[]
raw_text =""
stop_words = tuple(set(stopwords.words("english")))

def lemmatization(token_sentence):
    res = []
    for word in token_sentence:
        lemmatizer.lemmatize(word)
        res.append(word)
    return res


def get_wordnet_pos(word):
    """Map POS tag to first character lemmatize() accepts"""
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

    return tag_dict.get(tag, wordnet.NOUN)


# Simple test
def input_test(classifier):
    rev = input("Enter joke: ")
    rev = re.sub('[^a-zA-Z]', ' ', rev).lower().split()
    rev = ' '.join([lemmatizer.lemmatize(word, get_wordnet_pos(word)) for word in rev])
    X = vectorizer.transform([rev]).toarray()

    print(X.shape)
    print(X)

    if(classifier.predict(X) == [1]):
        print("It's a joke!")
    else:
        print("It's not a joke!")


for i in range(0,8000):
    
   humor_sentence = dataset['text'][i]

   #Tokenization by words or sentences
   humor_sentence = word_tokenize(humor_sentence)
   #remove stopwords and lemmatization with pos_tag
   humor_sentence = " ".join([lemmatizer.lemmatize(word, get_wordnet_pos(word)) for word in humor_sentence if word not in set(stopwords.words('english'))])  
  
      
   # remove alpha chars/not alphabetical caracters 
   humor_sentence2 = re.sub('[^a-zA-Z]', ' ', dataset['text'][i])
   
   humor_sentence2 = humor_sentence2.lower()
   humor_sentence2 = word_tokenize(humor_sentence2)
   humor_sentence2 = " ".join([lemmatizer.lemmatize(word, get_wordnet_pos(word)) for word in humor_sentence2 if word not in set(stopwords.words('english'))])  
      
   #raw_text += aux_reader+ "\n"
   
    
   
   data_list.append(humor_sentence)
   data_list2.append(humor_sentence2) 


In [2]:
# Create bag-of-words model
from sklearn.feature_extraction.text import CountVectorizer


vectorizer = CountVectorizer(max_features = 8000)
X = vectorizer.fit_transform(data_list).toarray()
y = dataset.iloc[:,2].values


print(vectorizer.get_feature_names())
print(X.shape, y.shape)

['00', '000', '03', '10', '100', '1000', '100th', '10k', '10yr', '11', '11th', '12', '1200', '13', '14', '15', '150', '153', '16', '17', '1704', '1759', '1760', '1776', '1789', '179', '17th', '18', '180', '1800', '1803', '1807', '1835', '1836', '1847', '1849', '1851', '1861', '1872', '1878', '1880', '1880s', '1890', '1890s', '19', '1900s', '1908', '1910â', '1911', '1921', '1922', '1924', '1928', '1930', '1940', '1945', '1948', '1950s', '1953', '1963', '1967', '1969', '1970s', '1975', '1977', '1981', '1984', '1985', '1991', '1992', '1995', '1996', '1998', '1st', '20', '200', '2000', '2001', '2003', '2006', '2007', '200thepisode', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '21', '22', '23', '239', '24', '25', '26', '28', '29', '2nd', '2x', '30', '300', '30pm', '31', '32', '33', '34', '35', '36', '360', '3629', '37', '3d', '3rd', '40', '400', '41', '42', '43', '45', '46', '48', '49', '4th', '50', '500', '51', '52', '55', '56', '57', '5am', '5pm

In [3]:
# Split dataset into training and test sets

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.30, random_state = 0)


print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)

(5600, 8000) (5600,)
(2400, 8000) (2400,)


In [4]:
# Fit Naive Bayes to the training set

from sklearn.naive_bayes import GaussianNB

classifier = GaussianNB()
classifier.fit(X_train, y_train)

# Predict test set results
y_pred = classifier.predict(X_test)

print(y_pred)


[0 1 0 ... 0 1 1]


In [5]:
# Generate metrics

from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score

# confusion matrix
print(confusion_matrix(y_test, y_pred))

# accuracy
print('Accuracy: ', accuracy_score(y_test, y_pred))

# precision
print('Precision: ', precision_score(y_test, y_pred))

# recall
print('Recall: ', recall_score(y_test, y_pred))

# f1
print('F1: ', f1_score(y_test, y_pred))

[[749 153]
 [606 892]]
Accuracy:  0.68375
Precision:  0.8535885167464115
Recall:  0.595460614152203
F1:  0.7015336217066458


In [6]:
input_test(classifier)


Enter joke: Hi
(1, 8000)
[[0 0 0 ... 0 0 0]]
It's not a joke!


In [8]:
# SVM

from sklearn.svm import SVC

classifier = SVC()
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)

print(confusion_matrix(y_test, y_pred))
print('Accuracy: ', accuracy_score(y_test, y_pred))
print('Precision: ', precision_score(y_test, y_pred))
print('Recall: ', recall_score(y_test, y_pred))
print('F1: ', f1_score(y_test, y_pred))

[[ 701  201]
 [ 191 1307]]
Accuracy:  0.8366666666666667
Precision:  0.866710875331565
Recall:  0.8724966622162884
F1:  0.869594145043247


In [65]:
input_test(classifier)

Enter joke: I once toasted the bride and groom at a Pakistani wedding. All I did was push the button on the drone
(1, 8000)
[[0 0 0 ... 0 0 0]]
It's not a joke!


In [67]:
from sklearn.linear_model import LogisticRegression

classifier = LogisticRegression()
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)

print(confusion_matrix(y_test, y_pred))
print('Accuracy: ', accuracy_score(y_test, y_pred))
print('Precision: ', precision_score(y_test, y_pred))
print('Recall: ', recall_score(y_test, y_pred))
print('F1: ', f1_score(y_test, y_pred))

[[480 129]
 [123 868]]
Accuracy:  0.8425
Precision:  0.8706118355065195
Recall:  0.875882946518668
F1:  0.8732394366197184


In [68]:
input_test(classifier)

Enter joke: I once toasted the bride and groom at a Pakistani wedding. All I did was push the button on the drone
(1, 8000)
[[0 0 0 ... 0 0 0]]
It's not a joke!


In [69]:
from sklearn.neural_network import MLPClassifier

classifier = MLPClassifier()
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)

print(confusion_matrix(y_test, y_pred))
print('Accuracy: ', accuracy_score(y_test, y_pred))
print('Precision: ', precision_score(y_test, y_pred))
print('Recall: ', recall_score(y_test, y_pred))
print('F1: ', f1_score(y_test, y_pred))

[[479 130]
 [145 846]]
Accuracy:  0.828125
Precision:  0.8668032786885246
Recall:  0.8536831483350151
F1:  0.8601931875953227


In [71]:
input_test(classifier)

Enter joke: I once toasted the bride and groom at a Pakistani wedding. All I did was push the button on the drone
(1, 8000)
[[0 0 0 ... 0 0 0]]
It's not a joke!


In [72]:
from sklearn.neural_network import BernoulliRBM
lassifier = BernoulliRBM()
classifier.fit(X_train, y_train)
y_pred = classifier.predict(X_test)

print(confusion_matrix(y_test, y_pred))
print('Accuracy: ', accuracy_score(y_test, y_pred))
print('Precision: ', precision_score(y_test, y_pred))
print('Recall: ', recall_score(y_test, y_pred))
print('F1: ', f1_score(y_test, y_pred))

[[479 130]
 [145 846]]
Accuracy:  0.828125
Precision:  0.8668032786885246
Recall:  0.8536831483350151
F1:  0.8601931875953227


In [73]:
input_test(classifier)

Enter joke: I once toasted the bride and groom at a Pakistani wedding. All I did was push the button on the drone
(1, 8000)
[[0 0 0 ... 0 0 0]]
It's not a joke!
